In [1]:
import inspect

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold

from metric.compute_metric import compute_clear
from utils import make_pred, save_data, iter_frame_sync
from metric.read_file import read_file
from utils import Point, mult_load, compare_corp_son
import numpy as np
from utils import compute_error


In [4]:
from utils import iter_frame, Corp
from utils.identity_assignation import identity_assignation


class InertieClasificator(BaseEstimator, ClassifierMixin):
    def __init__(self, coef_ine_hd = 1, coef_ine_bd =1, coef_dist_hd = 1, coef_dist_bd = 1, 
                 coef_size_hd = 1, coef_size_bd = 1, coef_pt_cache = 10, mem_length= 20, coef_col = 1,
                 seuil_parole = 60, bonus_parole = 10):
        
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        values.pop("self")
        self.seuil_parole = seuil_parole
        self.bonus_parole = bonus_parole

        for arg, val in values.items():
            setattr(self, arg, val)
    
    def fit(self, X, y=None):
        # on n'apprend rien pour l'instant
        pass
    
    def mk_dist(self):
        def fdist(id, corp):
            var = id.comp(corp)
            dist = var[0] * self.coef_ine_hd
            dist += var[1] * self.coef_ine_bd
            dist += var[2] * self.coef_col
            dist += var[3] * self.coef_dist_hd
            dist += var[4] * self.coef_dist_bd
            dist += var[5] * self.coef_pt_cache
            return dist
        return fdist
               
    
    def predict(self, X, y=None):
        # prediction
        
        def func (data):
            data_pos, data_son = data
            identites = dict()
            for frame_pos, frame_son in iter_frame_sync(data_pos,data_son):
                
          
                identites = identity_assignation(frame_pos, identites, None, self.mk_dist())
    
                for i, l in enumerate(frame_pos):
                    cp = Corp(l)
                    l = [Point(x[2], x[1]) for x in frame_son]
                    pson = min(l, key=lambda p:cp.hd_bary.dist(p))    
                    
                    id = None
                    for _id in identites.keys():
                        if _id == cp:
                            id = _id
                    parle = False
                    # print(str(Point(son[2], son[1])) + "vs " + str(cp.hd_bary))                    
                    if cp.hd_bary.dist(pson) < self.seuil_parole + int(id.get_speak())*self.bonus_parole :
                        # print("Match !! ")
                        parle = True
                    id.set_speak(parle)
                    yield (identites[cp], int(parle))  
        
        return [make_pred(vid[0], func, data_son=vid[1]) for vid in X]
    
    def score(self, X, y, sample_weight=None):
        pred = self.predict(X)
        res = [0, 0, 0, 0]
        for vid, raw in zip(pred, y):
            c = compute_error(raw, vid, 1)
            c = list(c)
            res = [a+b for a,b in zip(res, c)]
        # print(res/len(y))
        return [x/len(y) for x in res]
           
        

In [15]:
PATH = "data/Scenario03-05/"
data_ref = read_file(PATH+"groundTruth.txt")
data = np.loadtxt(PATH+"augmented_data.txt")
data_son = np.loadtxt(PATH+"mixture")
est = InertieClasificator(seuil_parole=60, bonus_parole=10)
res = est.predict([(data, data_son)], [data_ref])
print(est.score([(data, data_son)], [data_ref]))
save_data(res[0], PATH +  "res_tmp.txt")

n  :5860
f_neg : 0.0962457337884
f_pos : 0.103412969283
SA : 0.800341296928
[0.98476383951244284, 0.0, 0.80034129692832767, 0.98476383951244284]


In [3]:
data_ref, data = mult_load(["data/Scenario03-02/", "data/Scenario01-01/", "data/Scenario02-03/", "data/Scenario02-01/",
                            "data/Scenario02-02/", "data/Scenario03-03/", "data/Scenario03-02/", "data/Scenario03-01/",
                            "data/Scenario03-05/"], son=True, ficher_son="mixture")
est = InertieClasificator()
print(est.score(data, data_ref))

n  :4780
f_neg : 0.078870292887
f_pos : 0.175523012552
SA : 0.745606694561
n  :573
f_neg : 0.150087260035
f_pos : 0.00174520069808
SA : 0.848167539267


n  :728
f_neg : 0.221153846154
f_pos : 0.101648351648
SA : 0.677197802198


n  :1609
f_neg : 0.220012430081
f_pos : 0.0205096333126
SA : 0.759477936607


n  :1524
f_neg : 0.166666666667
f_pos : 0.0236220472441
SA : 0.809711286089


n  :1413
f_neg : 0.0396319886766
f_pos : 0.0176928520878
SA : 0.942675159236


n  :4780
f_neg : 0.078870292887
f_pos : 0.175523012552
SA : 0.745606694561


n  :7661
f_neg : 0.16042292129
f_pos : 0.0708784753949
SA : 0.768698603315


n  :5860
f_neg : 0.0962457337884
f_pos : 0.103412969283
SA : 0.800341296928
[0.97933572191362606, 0.0, 0.78860922364016373, 0.97933572191362606]


3-2
n  :4780
f_neg : 0.078870292887
f_pos : 0.175523012552

1-1
n  :573
f_neg : 0.150087260035
f_pos : 0.00174520069808

2-3
n  :728
f_neg : 0.221153846154
f_pos : 0.101648351648

2-1
n  :1609
f_neg : 0.220012430081
f_pos : 0.0205096333126

2-2
n  :1524
f_neg : 0.166666666667
f_pos : 0.0236220472441

3-3
n  :1413
f_neg : 0.0396319886766
f_pos : 0.0176928520878

3-2
n  :4780
f_neg : 0.078870292887
f_pos : 0.175523012552

3-1
n  :7661
f_neg : 0.16042292129
f_pos : 0.0708784753949

3-5
n  :5860
f_neg : 0.0962457337884
f_pos : 0.103412969283

[0.97933572191362606, 0.0, 0.78860922364016373, 0.97933572191362606]
